<a href="https://colab.research.google.com/github/mrdbourke/tensorflow-deep-learning/blob/main/video_notebooks/08_introduction_to_nlp_in_tensorflow_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving informaton out of natural language (could be seqeuences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

> 📖 **Resource:** See all course materials, resources and extra-curriculum for this notebook on GitHub: https://github.com/mrdbourke/tensorflow-deep-learning/

## Check for GPU

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ffcdd345-2451-5781-5b76-e477b5a7568d)


## Get helper functions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-08-15 12:17:22--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-15 12:17:22 (98.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster).

See the original source here: https://www.kaggle.com/c/nlp-getting-started

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-08-15 12:17:24--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.135.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.135.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-08-15 12:17:24 (144 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/

But I prefer to get visual straight away.

So another way to do this is to use pandas...

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) 
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real diaster)
Text:
Raw Video: Dust Storm Rolls Through Texas http://t.co/QllkOfdyzX http://t.co/rGjJuMnNah

---

Target: 1 (real diaster)
Text:
@greateranglia I know the cow incident not yr fault by how where they on the line could of caused a derailment

---

Target: 1 (real diaster)
Text:
1.9 earthquake occurred 15km E of Anchorage Alaska at 00:11 UTC! #earthquake #Anchorage http://t.co/QFyy5aZIFx

---

Target: 1 (real diaster)
Text:
#IDFire Parker Ridge Fact Sheet Aug 6 2015 (Parker Ridge Wildfire): Information Phone: 208-267-6773 Email: pa... http://t.co/ZggpaCjP7D

---

Target: 0 (not real diaster)
Text:
Thief Broke Front Window Of Hicksville Store Stole 50 Cell Phones; Fled Crashed Into... http://t.co/6odNBttPSq

---



### Split data into training and validation sets

We want to be able to see how our model is performing on unseen data whilst it trains.

And because the testing dataset doesn't have labels, we'll have to create a validation dataset to evaluate on (the model won't see the validation dataset during training so we can use its samples and labels to evaluate our model's performance).

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=42)

In [ ]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
# Check the first 10 samples 
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [ ]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)

In [ ]:
len(train_sentences[0].split())

7

In [ ]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
# Fit the text vectorizer instance to the training data using the adapt() method
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
text_vectorizer([random_sentence])

Oh the usual. Mass murder and world domination plans over coffee. How's your day going?


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 392,    2, 2238,  157,  315,    7,  107,    1,  645,   60, 1542,
           1,   33,  101,  104]])>

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
top_5_words

['', '[UNK]', 'the', 'a', 'in']

In [ ]:
bottom_5_words = words_in_vocab[-5:]
bottom_5_words

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

##Creating an embedding using embedding layer

In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length)

In [ ]:
random_sentence = random.choice(train_sentences)
print(random_sentence)

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

NHS England announces new plan to meet emergency care targets http://t.co/0x2BIEqXPV


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00451935,  0.03723775, -0.03475238, ...,  0.02113981,
          0.00747805, -0.04684435],
        [-0.03130349, -0.03065364, -0.01378814, ..., -0.01232495,
         -0.04042579,  0.00072727],
        [ 0.01726986,  0.03136216, -0.01655392, ..., -0.03045288,
          0.00338695,  0.0399174 ],
        ...,
        [ 0.00434946, -0.04079541,  0.04726494, ...,  0.00665817,
         -0.01822245,  0.02419044],
        [ 0.00434946, -0.04079541,  0.04726494, ...,  0.00665817,
         -0.01822245,  0.02419044],
        [ 0.00434946, -0.04079541,  0.04726494, ...,  0.00665817,
         -0.01822245,  0.02419044]]], dtype=float32)>

#Different experimental models
1. Naive baeys baseline model
2. feed forward neural network (dense network)
3. LSTM model
4. GRU model
5. bidirectional LSTM model
6. 1D convolutional neural netwokr
7. tfhub pretrained model(transfer learning)
8. same as above but with 10% of input data

##Model 1 naive bayes baseline model
using multinomial naive bayes

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf",TfidfVectorizer()),
                    ("clf",MultinomialNB())
])

model_0.fit(train_sentences,train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
baseline_score = model_0.score(val_sentences,val_labels)
print("Out base line model achieves an accuracy of",baseline_score*100)

Out base line model achieves an accuracy of 79.26509186351706


In [ ]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:5]

array([1, 1, 1, 0, 0])

###Creating an evaluation function for our model experiments


In [ ]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred)
  model_results = {"accuracy":model_accuracy,
                   "precision":model_precision,
                   "recall":model_recall,
                   "f1":model_f1}
  return model_results

In [ ]:
baseline_results = calculate_results(val_labels,baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'f1': array([0.83010753, 0.73400673]),
 'precision': array([0.74806202, 0.88617886]),
 'recall': array([0.93236715, 0.62643678])}

###Model 1: A simple dense model

In [ ]:
from helper_functions import create_tensorboard_callback

save_dir = "model_logs"

In [ ]:
#building model using functional api

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),
                     dtype = tf.string)
x = text_vectorizer(inputs) #turn thee input text into numbers
x = embedding(x)#turn numbers into embedding
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")


In [ ]:
model_1.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_1.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(dir_name=save_dir,
                                                   experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210815-121733
Epoch 1/5
215/215 [==============================] - 7s 16ms/step - loss: 0.6125 - accuracy: 0.6938 - val_loss: 0.5388 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4445 - accuracy: 0.8154 - val_loss: 0.4754 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3495 - accuracy: 0.8610 - val_loss: 0.4580 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2869 - accuracy: 0.8904 - val_loss: 0.4632 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2395 - accuracy: 0.9132 - val_loss: 0.4808 - val_accuracy: 0.7861


In [ ]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4808 - accuracy: 0.7861


[0.4807831645011902, 0.7860892415046692]

In [ ]:
model_1_pred_prob = model_1.predict(val_sentences)
model_1_pred_prob[0]

array([0.34694064], dtype=float32)

In [ ]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_prob))

In [ ]:
model_1_results = calculate_results(val_labels, model_1_preds)
model_1_results

{'accuracy': 0.7860892388451444,
 'f1': array([0.81828317, 0.7400319 ]),
 'precision': array([0.75983437, 0.83154122]),
 'recall': array([0.88647343, 0.66666667])}

In [ ]:
baseline_results

{'accuracy': 0.7926509186351706,
 'f1': array([0.83010753, 0.73400673]),
 'precision': array([0.74806202, 0.88617886]),
 'recall': array([0.93236715, 0.62643678])}

##Visualizing learned embeddings

In [ ]:
words_in_vocab

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'be',
 'are',
 'was',
 'have',
 'like',
 'as',
 'up',
 'so',
 'just',
 'but',
 'me',
 'im',
 'your',
 'not',
 'amp',
 'out',
 'its',
 'will',
 'an',
 'no',
 'has',
 'fire',
 'after',
 'all',
 'when',
 'we',
 'if',
 'now',
 'via',
 'new',
 'more',
 'get',
 'or',
 'about',
 'what',
 'he',
 'people',
 'news',
 'been',
 'over',
 'one',
 'how',
 'dont',
 'they',
 'who',
 'into',
 'were',
 'do',
 'us',
 '2',
 'can',
 'video',
 'emergency',
 'there',
 'disaster',
 'than',
 'police',
 'would',
 'his',
 'still',
 'her',
 'some',
 'body',
 'storm',
 'crash',
 'burning',
 'suicide',
 'back',
 'man',
 'california',
 'why',
 'time',
 'them',
 'had',
 'buildings',
 'rt',
 'first',
 'cant',
 'see',
 'got',
 'day',
 'off',
 'our',
 'going',
 'nuclear',
 'know',
 'world',
 'bomb',
 'fires',
 'love',
 'killed',
 'go',
 'attack',
 'youtube',
 'dead

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [ ]:
#model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#get the weight matrix of embedding layer
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [ ]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

## Recurrent Neural Networks(RNN's)

its useful for sequence data



###Model 2:LSTM

LSTM = long short term memory(one fo the most popular)

input->tokenize->embedding>layers(RNNs/dense)->output(label probability)

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype = "string")
x= text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64, return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [ ]:
model_2.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(save_dir,
                                                                    "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210815-121757
Epoch 1/5
215/215 [==============================] - 11s 24ms/step - loss: 0.2207 - accuracy: 0.9226 - val_loss: 0.5359 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1599 - accuracy: 0.9406 - val_loss: 0.6977 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1303 - accuracy: 0.9511 - val_loss: 0.6789 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1050 - accuracy: 0.9580 - val_loss: 0.8261 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0849 - accuracy: 0.9672 - val_loss: 1.0760 - val_accuracy: 0.7782


In [ ]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[-10:]

array([[9.9998593e-01],
       [3.0501635e-04],
       [9.9996173e-01],
       [9.9508190e-01],
       [2.6249707e-02],
       [5.5162199e-02],
       [4.0311202e-02],
       [8.6547726e-01],
       [1.9815534e-02],
       [1.1758953e-04]], dtype=float32)

In [ ]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [ ]:
model_2_results = calculate_results(val_labels,
                                    model_2_preds)

In [ ]:
model_2_results

{'accuracy': 0.7782152230971129,
 'f1': array([0.80462428, 0.74355083]),
 'precision': array([0.77161863, 0.78778135]),
 'recall': array([0.84057971, 0.70402299])}

In [ ]:
baseline_results

{'accuracy': 0.7926509186351706,
 'f1': array([0.83010753, 0.73400673]),
 'precision': array([0.74806202, 0.88617886]),
 'recall': array([0.93236715, 0.62643678])}

###Model 3 :GRU

Another popular and effective RNN component is gru or gated recurrent network

In [ ]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.GRU(64,return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(64,return_sequences=True)(x)
print(x.shape)
x = layers.GRU(64)(x)
print(x.shape)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name="model_3_GRU")

(None, 15, 128)
(None, 15, 64)
(None, 15, 64)
(None, 64)


In [ ]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 15, 64)            37248     
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 64)            33024     
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24960     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                

In [ ]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 8s 23ms/step - loss: 0.1416 - accuracy: 0.9518 - val_loss: 0.9233 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0755 - accuracy: 0.9699 - val_loss: 0.8869 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0618 - accuracy: 0.9730 - val_loss: 1.1575 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0578 - accuracy: 0.9747 - val_loss: 1.3995 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0596 - accuracy: 0.9736 - val_loss: 1.2494 - val_accuracy: 0.7717


In [ ]:
model_3_history

###Model 4: Bidirectional RNN

normal rnn goes from left to right but bidirectional (as the name sya enough) it goes left to right as well as right to left

In [ ]:
# building it 
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.GRU(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs,name="model_4")

In [ ]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [ ]:
model_4.fit(train_sentences,
            train_labels,
            epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(save_dir,
                                                   "model_4")])

Saving TensorBoard log files to: model_logs/model_4/20210815-121850
Epoch 1/5
215/215 [==============================] - 11s 31ms/step - loss: 0.1138 - accuracy: 0.9610 - val_loss: 0.7635 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0670 - accuracy: 0.9737 - val_loss: 1.0365 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0495 - accuracy: 0.9780 - val_loss: 1.3188 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0504 - accuracy: 0.9766 - val_loss: 1.3604 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0412 - accuracy: 0.9794 - val_loss: 1.4549 - val_accuracy: 0.7493


In [ ]:
model_4_pred_probs = model_4.predict(val_sentences)

In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [ ]:
model_4_results = calculate_results(val_labels,
                                    model_4_preds)

In [ ]:
model_4_results

{'accuracy': 0.7493438320209974,
 'f1': array([0.77070828, 0.723589  ]),
 'precision': array([0.76610979, 0.72886297]),
 'recall': array([0.77536232, 0.7183908 ])}

###Model 4: 1D CNN convolution neural network

before we used it to play with images which is basically 2D but here we will use use Conv1D because our text is 1D

input->tokenization->embedding->layer(con1d+pooling)->outputs

In [ ]:
embedding_test = embedding(text_vectorizer(["this is lola lallan"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="valid")
conv_1d_output = conv_1d(embedding_test)#passing the test embedding through the conv1D lyer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)# get the  feature with the highest value

embedding_test.shape,conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [ ]:
inputs = layers.Input(shape=(1,),dtype=tf.string)

x = embedding(text_vectorizer(inputs))

x = layers.Conv1D(filters=64,
                  kernel_size = 5,
                  strides=1,
                  activation="relu",
                  padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1,activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs,outputs,name="model_5_conv1D")

model_5.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5.summary()

Model: "model_5_conv1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 11, 64)            41024     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 1,321,089
Trainable params: 1,321,089
Non-trainable params: 0
__________________________________________

In [ ]:
model_5.history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data = (val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(save_dir,
                                                                     "model_5")])

Saving TensorBoard log files to: model_logs/model_5/20210815-121945
Epoch 1/5
215/215 [==============================] - 6s 18ms/step - loss: 0.1287 - accuracy: 0.9591 - val_loss: 0.8527 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0746 - accuracy: 0.9714 - val_loss: 1.0268 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0606 - accuracy: 0.9759 - val_loss: 1.1278 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0551 - accuracy: 0.9774 - val_loss: 1.1965 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0507 - accuracy: 0.9777 - val_loss: 1.1998 - val_accuracy: 0.7598


In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs

array([[3.79487187e-01],
       [9.54686165e-01],
       [9.99829054e-01],
       [2.81389002e-02],
       [1.00381226e-06],
       [9.89360452e-01],
       [8.75649273e-01],
       [9.99954224e-01],
       [9.99999404e-01],
       [6.15010798e-01],
       [2.86678755e-07],
       [8.69943082e-01],
       [1.02815093e-05],
       [2.29316711e-01],
       [1.33221715e-06],
       [2.00079996e-02],
       [1.30555069e-03],
       [9.81802714e-06],
       [6.35905005e-03],
       [9.99382377e-01],
       [8.79668832e-01],
       [1.37395068e-06],
       [9.99822676e-01],
       [6.71723625e-04],
       [9.99988556e-01],
       [1.00000000e+00],
       [1.29212349e-04],
       [2.87840609e-04],
       [9.83760328e-05],
       [5.71585476e-01],
       [3.17132026e-01],
       [4.24876052e-05],
       [1.91149697e-01],
       [4.14095633e-03],
       [3.45243476e-02],
       [6.95243895e-01],
       [9.99998927e-01],
       [9.98291746e-02],
       [9.05097369e-03],
       [9.98899698e-01],


In [ ]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

In [ ]:
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
model_5_results = calculate_results(val_labels,
                                    model_5_preds)

In [ ]:
model_5_results

{'accuracy': 0.7598425196850394,
 'f1': array([0.78794902, 0.72314675]),
 'precision': array([0.75723831, 0.76357827]),
 'recall': array([0.82125604, 0.68678161])}

##Model 6: Using transfer earning(using USE feature extractor)

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")


In [ ]:
#creating model using sequential api

model_6 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(128,activation="relu"),
  layers.Dense(128,activation="relu"),
  layers.Dense(1,activation="sigmoid")
],name="model_6_USE")

model_6.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_7 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 129       
Total params: 256,880,129
Trainable params: 82,305
Non-trainable params: 256,797,824
_________________________________________________________________


In [ ]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(save_dir,
                                                                     "model_6_use")])

Saving TensorBoard log files to: model_logs/model_6_use/20210815-122022
Epoch 1/10
215/215 [==============================] - 7s 23ms/step - loss: 0.4732 - accuracy: 0.7827 - val_loss: 0.4401 - val_accuracy: 0.8071
Epoch 2/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3922 - accuracy: 0.8307 - val_loss: 0.4226 - val_accuracy: 0.8136
Epoch 3/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3643 - accuracy: 0.8437 - val_loss: 0.4241 - val_accuracy: 0.8189
Epoch 4/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3277 - accuracy: 0.8631 - val_loss: 0.4445 - val_accuracy: 0.8176
Epoch 5/10
215/215 [==============================] - 2s 10ms/step - loss: 0.2815 - accuracy: 0.8867 - val_loss: 0.4520 - val_accuracy: 0.8136
Epoch 6/10
215/215 [==============================] - 2s 11ms/step - loss: 0.2284 - accuracy: 0.9118 - val_loss: 0.5658 - val_accuracy: 0.8031
Epoch 7/10
215/215 [==============================] - 2s 10ms/step - l

In [ ]:
model_6_pred_probs = model_6.predict(val_sentences)

In [ ]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))

In [ ]:
model_6_results = calculate_results(val_labels,
                                    model_6_preds)

In [ ]:
model_6_results

{'accuracy': 0.7992125984251969,
 'f1': array([0.83278689, 0.74876847]),
 'precision': array([0.76047904, 0.87356322]),
 'recall': array([0.92028986, 0.65517241])}

##Model 7: TF hub pretrained USE but with only 10% of the training data

In [ ]:
#getting 10% of the data

train_10_percent = train_df_shuffled[["text","target"]].sample(frac=0.1,random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()

In [ ]:
# making a better 10% training data set
train_10_percent_split = int(0.1*len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]
len(train_sentences_10_percent),len(train_labels_10_percent)

(685, 685)

In [ ]:
model_7 = tf.keras.models.clone_model(model_6)

model_7.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [ ]:
model_7.fit(train_sentences_10_percent,
            train_labels_10_percent,
            epochs=5,
            validation_data=(val_sentences,val_labels),
            callbacks=[create_tensorboard_callback(save_dir,
                                                   "model_7")])

Saving TensorBoard log files to: model_logs/model_7/20210815-124720
Epoch 1/5
22/22 [==============================] - 1s 56ms/step - loss: 0.2237 - accuracy: 0.9372 - val_loss: 0.5758 - val_accuracy: 0.7520
Epoch 2/5
22/22 [==============================] - 0s 19ms/step - loss: 0.1879 - accuracy: 0.9533 - val_loss: 0.6202 - val_accuracy: 0.7480
Epoch 3/5
22/22 [==============================] - 0s 19ms/step - loss: 0.1432 - accuracy: 0.9679 - val_loss: 0.6656 - val_accuracy: 0.7520
Epoch 4/5
22/22 [==============================] - 0s 19ms/step - loss: 0.1096 - accuracy: 0.9752 - val_loss: 0.7285 - val_accuracy: 0.7533
Epoch 5/5
22/22 [==============================] - 0s 19ms/step - loss: 0.0783 - accuracy: 0.9869 - val_loss: 0.8122 - val_accuracy: 0.7559


In [ ]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))


In [ ]:
model_7_results = calculate_results(val_labels,
                                    model_7_preds)

In [ ]:
# highly efficient than training on more data because it is going to reduce the overfitting 
model_7_results

{'accuracy': 0.7559055118110236,
 'f1': array([0.79194631, 0.7047619 ]),
 'precision': array([0.7375    , 0.78723404]),
 'recall': array([0.85507246, 0.63793103])}

In [ ]:
model_1_results

{'accuracy': 0.7860892388451444,
 'f1': array([0.81828317, 0.7400319 ]),
 'precision': array([0.75983437, 0.83154122]),
 'recall': array([0.88647343, 0.66666667])}

In [ ]:
# data leakage problem is cause because we are using the 10% of the shuffled training data set so it means that the validation data may contain some of
# some of the cases from the 10% training data set

In [ ]:
model_6_results

{'accuracy': 0.7992125984251969,
 'f1': array([0.83278689, 0.74876847]),
 'precision': array([0.76047904, 0.87356322]),
 'recall': array([0.92028986, 0.65517241])}

In [ ]:
model_6.save("model_6.h5")

In [ ]:
# to load a model which has some layer from a pretrained model from tensorflow hub then we need to create custom objects 
loaded_model = tf.keras.models.load_model("model_6.h5",
                                          custom_objects={"KerasLayer":hub.KerasLayer})

In [ ]:
loaded_model.evaluate(val_sentences,val_labels)

24/24 [==============================] - 1s 9ms/step - loss: 0.8746 - accuracy: 0.7992


[0.8745608329772949, 0.7992125749588013]

In [ ]:
model_6_results

{'accuracy': 0.7992125984251969,
 'f1': array([0.83278689, 0.74876847]),
 'precision': array([0.76047904, 0.87356322]),
 'recall': array([0.92028986, 0.65517241])}

## Finding the most wrong examples

In [ ]:
val_df = pd.DataFrame({"Text":val_sentences,
                       "Target":val_labels,
                       "Prediction":model_6_preds,
                       "Prediction Probability":tf.squeeze(model_6_pred_probs)})


In [ ]:
val_df.head()

,Text,Target,Prediction,Prediction Probability
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.001329
1,FedEx no longer to transport bioterror germs i...,0,0.0,0.038868
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.999999
3,@camilacabello97 Internally and externally scr...,1,0.0,0.056656
4,Radiation emergency #preparedness starts with ...,1,0.0,0.203175


In [ ]:
most_wrong = val_df[val_df["Target"]!=val_df["Prediction"]].sort_values("Prediction Probability",ascending=False)

In [ ]:
most_wrong.head()

,Text,Target,Prediction,Prediction Probability
381,Deaths 3 http://t.co/nApviyGKYK,0,1.0,0.999877
209,Ashes 2015: AustraliaÛªs collapse at Trent Br...,0,1.0,0.998009
628,@noah_anyname That's where the concentration c...,0,1.0,0.996252
31,? High Skies - Burning Buildings ? http://t.co...,0,1.0,0.994905
49,@madonnamking RSPCA site multiple 7 story high...,0,1.0,0.993426


In [ ]:
most_wrong.tail()

,Text,Target,Prediction,Prediction Probability
408,@willienelson We need help! Horses will die!Pl...,1,0.0,0.000061
361,Next May I'll be free...from school from oblig...,1,0.0,0.000042
373,Plans by former First Lady and wife of ex-Pres...,1,0.0,0.000027
221,going to redo my nails and watch behind the sc...,1,0.0,0.000010
262,Just came back from camping and returned with ...,1,0.0,0.000007
